In [1]:
import pandas as pd
import networkx as nx
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
from torch_geometric.nn import GCNConv
from itertools import product

# Load data
signal_df = pd.read_csv('Dijet_bb_pt10_15_dw.csv')
background_df = pd.read_csv('Dijet_qq_pt10_15_dw.csv')

create empty tensor: edge_index = torch.tensor([[], []], dtype=torch.long)

In [49]:
# Fully Connected Edges
def create_graph_from_df(df):
    G = nx.Graph()
    for node in G.nodes(): 
        node_features = df[df['Jet0_FD_OWNPV'] == node].iloc[:, 2:].values
        if len(node_features) > 0:
            G.nodes[node]['x'] = torch.tensor(node_features[0], dtype=torch.float)
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    G.add_edge(nodes[i], nodes[j])
        else:
            G.nodes[node]['x'] = torch.tensor([0.0] * (df.shape[1] - 2), dtype=torch.float)
            for i in range(len(nodes)):
                for j in range(i + 1, len(nodes)):
                    G.add_edge(nodes[i], nodes[j])
    data = from_networkx(G)
    return data

In [50]:
signal_graph = create_graph_from_df(signal_df)
background_graph = create_graph_from_df(background_df)

In [51]:
# Combine signal and background graphs
def create_combined_dataset(signal_graph, background_graph):
    signal_graph.y = torch.ones(signal_graph.num_nodes, dtype=torch.long)
    background_graph.y = torch.zeros(background_graph.num_nodes, dtype=torch.long)
    signal_graph.train_mask = torch.ones(signal_graph.num_nodes, dtype=torch.bool)
    background_graph.train_mask = torch.ones(background_graph.num_nodes, dtype=torch.bool)
    signal_graph.test_mask = torch.ones(signal_graph.num_nodes, dtype=torch.bool)
    background_graph.test_mask = torch.ones(background_graph.num_nodes, dtype=torch.bool)
    combined_graph = Data(
        x=torch.cat([signal_graph.x, background_graph.x], dim=0),
        edge_index=torch.cat([signal_graph.edge_index, background_graph.edge_index + signal_graph.num_nodes], dim=1),
        y=torch.cat([signal_graph.y, background_graph.y], dim=0),
        train_mask=torch.cat([signal_graph.train_mask, background_graph.train_mask], dim=0),
        test_mask=torch.cat([signal_graph.test_mask, background_graph.test_mask], dim=0),
    )
    return combined_graph

combined_graph = create_combined_dataset(signal_graph, background_graph)

TypeError: expected Tensor as element 0 in argument 0, but got NoneType

In [ ]:
# Define GNN model
class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels=signal_graph.num_node_features, out_channels=16)
        self.conv2 = GCNConv(in_channels=16, out_channels=2)
    
    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=1)

model = GNN()

In [ ]:
# Training parameters
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
def train(model, data, optimizer, criterion):
    model.train()
    optimizer.zero_grad()
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return loss.item()

for epoch in range(1):
    loss = train(model, combined_graph, optimizer, criterion)
    print(f'Epoch {epoch}, Loss: {loss}')

# Evaluation
def test(model, data):
    model.eval()
    _, pred = model(data).max(dim=1)
    correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    acc = correct / data.test_mask.sum().item()
    return acc

accuracy = test(model, combined_graph)
print(f'Accuracy: {accuracy}')